***
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates the usage of K8S kubectl action.
    Using this action, we can query the K8S cluster and find out pod(s) stuck in ImagePullBackOff state.
    </b>
</div>

<br>
<a href="https://us.app.unskript.io/profiles/a53b5860500e142aa387ce55d5e85f139596c521dfb5c920cc2bc47c38fc0b11">Open in unSkript</a>

<center><h2>K8S Pod stuck in ImagePullBackOff State</h2></center>

An `ImagePullBackOff` error occurs when a Pod startup fails to pull the specified image. The reasons could be Non-Existent of the repository or Permission to Access the repository issues. This runbook helps to walk through the steps involved in debugging such a Pod. 

    The POD stuck in such a state generally is shown like this with kubectl command
    
       NAME                     READY     STATUS             RESTARTS   AGE
       nginx-7ef9efa7cd-qasd2   0/1       ImagePullBackOff   0          1m
    
   

### Initial Steps Overview
    1. Gather Information
    1.1 Create a list of Pods in ImagePullBackOff state
    1.2 Extract Events for the Pod
    2. Examine Events 
    3. Check Registry Accessibility
    
    
The Original Author of this Runbook is [Ian Miell](https://containersolutions.github.io/runbooks/posts/kubernetes/imagepullbackoff/)
***

### 1. Gather Information

We will use `kubectl get pods -n {namespace} | grep ImagePullBackOff` to find out all the PODs that are stuck in 
the `ImagePullBackOff` state for the given `namespace`. Please note, we will be using the unSkript's `kubectl lego` 
to accomplish this task. We then will store this list as `unhealthyPods`.  This covers Steps `1.0` and `1.1`.

In [20]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    if handle.client_side_validation != True:
        print(f"Connector for K8S is invalid {handle}")
        return str()
    
    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} | grep ImagePullBackOff | cut -d' ' -f 1 | tr -d ' '\\""
    }''')
task.configure(outputName="unhealthyPods")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 1.2 Extract Events for the Pod

Here we will iterate over the `unhealthyPods` list, use the unSkript `kubectl` Lego once again
to find out the `describe` the `unhealthyPods` and extract the `Events` section for those `pods`.

This Action performs the equivalent of  `kubectl describe pod -n {namespace} unhealthyPods`. 
And captures the `Events` section of the `kubectl describe pod` output. This output is stored 
in a variable `podEvents`.

In [27]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype

def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)

@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    if handle.client_side_validation != True:
        print(f"Connector for K8S is invalid {handle}")
        return str()
    
    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl describe pod {unhealthyPods.strip()} -n {namespace} | grep -A 10 Events\\""
    }''')
task.configure(outputName="podEvents")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, hdl=hdl, args=args, lego_printer=k8s_kubectl_command_printer)

### 2 Examine Events

This is an `unSkript custom cell` in this example, we show how we can process the `podEvents` 
from the Earlier action and identify possible cause fot he pod stuck in `ImagePullBackOff` state.

In [25]:
import re

"""
This Custom Action searches Known errors in the podEvents variable.
The well known errors are listed in the error_msgs variable. If
there is a new error message that was found, you can add it to this
list and the next run, the runbook will catch that error.
"""

def check_msg(msg):
    return re.search(msg, podEvents)

error_msgs = ["repository (.*) does not exist or no pull access",
              "manifest for (.*) not found",
              "pull access denied, repository does not exist or may require authorization",
             "Back-off pulling image (.*)"]
cause_found = False
result = ''
for err in error_msgs:
    result = check_msg(err)
    if result is not None:
        print("PROBABLE CAUSE: ", f"{result.string}")
        cause_found = True

repoLocation = ''
if cause_found is False:
    print("ERROR MESSAGE : \n", podEvents)
else:
    try:
        repoLocation = result.groups()[0]
    except:
        pass
    else:
        print("Image Repo Location : ", repoLocation)

### 3 Check Registry Accessibility

This once again uses `unSkript custom cell` to indicate the user to check if the Docker image registry
is accessible from the Pod.

In [26]:
from IPython.display import Markdown as md

if repoLocation is not None:
    display(md(f"**Please verify Repo {repoLocation} is accessible from the K8S POD**"))

### Conclusion

This Runbook demonstrated the ease of using unSkript Legos to construct a full fledged runbook that
is ready to be use. With simple Legos provided by unSkript, it is possible to construct complex workflows
to accomplish mission critical tasks. 

For more information about `unSkript` please visit us at `www.unskript.com`